In [1]:
import numpy as np
from numba import jit
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import pyvista as pv
from scipy import interpolate
import surface_normalize_ver2

In [2]:
import torch

def ray_point_cloud_intersection_torch(ray_origin, ray_direction, point_cloud, threshold=0.5):
    # 将数据转移到GPU
    ray_origin = torch.tensor(ray_origin, device='cuda')
    ray_direction = torch.tensor(ray_direction, device='cuda')
    point_cloud = torch.tensor(point_cloud, device='cuda')
    
    # 将光线方向标准化（单位向量）
    ray_direction = ray_direction / torch.norm(ray_direction)

    # 计算从光线起点到每个点的向量
    point_vectors = point_cloud - ray_origin

    # 计算点向量在光线方向上的投影长度
    t = torch.matmul(point_vectors, ray_direction)

    # 计算光线上投影点的坐标
    projections = ray_origin + torch.outer(t, ray_direction)

    # 计算当前点到投影点的距离
    distances = torch.norm(point_cloud - projections, dim=1)

    # 找到距离小于阈值的点
    indice = torch.where(distances < threshold)[0]
    
    if indice.numel() == 0:
        return None
    
    near_point = point_cloud[indice]
    distances_near_point = torch.norm(ray_origin - near_point, dim=1)

    # 找到最小距离和最近点
    min_index = torch.argmin(distances_near_point)
    closest_point = near_point[min_index]

    # 将结果从GPU传回到CPU
    return closest_point.cpu().numpy().astype(int)

In [3]:
def ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, point_cloud):
    # 将光线方向标准化（单位向量）
    ray_direction = ray_direction / np.linalg.norm(ray_direction)

    # 计算从光线起点到每个点的向量
    point_vectors = point_cloud - ray_origin

    # 计算点向量在光线方向上的投影长度
    t = np.dot(point_vectors, ray_direction)

    # 计算光线上投影点的坐标
    projections = ray_origin + np.outer(t, ray_direction)

    # 计算当前点到投影点的距离
    distances = np.linalg.norm(point_cloud - projections, axis=1)

    indice = np.where(distances < 0.5)[0]
    # 如果没有点满足条件，则返回空结果
    if indice.size == 0:
        return None
    near_point = point_cloud[indice]

    distances_near_point = np.linalg.norm(ray_origin - near_point, axis=1)
    # 找到最小距离和最近点
    min_index = np.argmin(distances_near_point)
    closest_point = point_cloud[indice[min_index]]
    # min_distance = distance_in[min_index]
    return closest_point.astype(int)

In [5]:
def scanZ2(film): # fast scanZ with NumPy
    # 将film的最后四个通道求和
    film_sum = np.sum(film[1:-1,1:-1,1:-1,6:], axis=-1)
    
    # 获取film的形状
    xshape, yshape, zshape = film_sum.shape
    
    # 初始化一个全零的表面稀疏数组
    surface_sparse = np.zeros((xshape, yshape, zshape), dtype=bool)
    
    # 当前平面的布尔索引
    current_plane = film_sum != 0
    
    # 获取周围邻居的布尔索引
    neighbors = np.zeros_like(film_sum, dtype=bool)
    
    neighbors[1:, :, :] |= film_sum[:-1, :, :] == 0  # 上面
    neighbors[:-1, :, :] |= film_sum[1:, :, :] == 0  # 下面
    neighbors[:, 1:, :] |= film_sum[:, :-1, :] == 0  # 左边
    neighbors[:, :-1, :] |= film_sum[:, 1:, :] == 0  # 右边
    neighbors[:, :, 1:] |= film_sum[:, :, :-1] == 0  # 前面
    neighbors[:, :, :-1] |= film_sum[:, :, 1:] == 0  # 后面
    
    # 获取满足条件的索引
    condition = current_plane & neighbors
    
    # 更新表面稀疏数组
    surface_sparse[condition] = True
    
    # 找到所有非零元素的索引，表示表面点的坐标
    surface = np.argwhere(surface_sparse)
    
    return surface + 1

In [7]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.2.2+cpu
False


In [6]:
import time
film = np.zeros((100, 100, 160, 8), dtype=int)

bottom = 10
film[:, :, 0:bottom, 6] = 30 # bottom

height = 40

# film[:, :19, 0:height] = 10
# film[:, 81:, 0:height] = 10
# film[:19, :, 0:height] = 10
# film[81:, :, 0:height] = 10

film[:, :17, 0:height, 6] = 30
film[:, 83:, 0:height, 6] = 30
film[:17, :, 0:height, 6] = 30
film[83:, :, 0:height, 6] = 30
ray_origin = np.array([10.3, 10.1, 160])
ray_direction = np.array([0, 0, -10])
surface = scanZ2(film)
closest_point = ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, surface)


# 运行次数
n_runs = 10

# 测试未使用 Numba 的耗时
start_time = time.time()
for _ in range(n_runs):
    film_sum_slow = ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, surface)
slow_duration = (time.time() - start_time) / n_runs
print(f"Without Numba: {slow_duration:.6f} seconds (average over {n_runs} runs)")

# 测试使用 Numba 的耗时
start_time = time.time()
for _ in range(n_runs):
    film_sum_fast = ray_point_cloud_intersection_torch(ray_origin, ray_direction, surface, threshold=0.5)
fast_duration = (time.time() - start_time) / n_runs
print(f"With Numba: {fast_duration:.6f} seconds (average over {n_runs} runs)")

# 确认两者计算结果是否相同
assert np.array_equal(film_sum_slow, film_sum_fast), "Results do not match!"

# 打印加速倍数
print(f"Numba speedup: {slow_duration / fast_duration:.2f}x")

Without Numba: 0.001721 seconds (average over 10 runs)


AssertionError: Torch not compiled with CUDA enabled

In [2]:
etching = np.load('./Data/For_etching_transport_TS60_deposit_0618_linear_paper_NoDSMC2_empty2.npy')

In [4]:
testpy = surface_normalize_ver2.surface_normal(center_with_direction=np.array([[50,50,10]]),range3D=np.array([[0, 100, 0, 100, 0, 150]]), InOrOut=[1],celllength=1e-9,  yield_hist=None)

In [5]:
film = np.zeros((20, 20, 20, 8), dtype=int)

film[:, :, :8, 6] = 30

box_half_size = 0.5
surface = testpy.get_pointcloud(film)

In [6]:
submesh = pv.PolyData(surface[50:60, 3:])
submesh["radius"] = np.ones(surface[50:60, 3:].shape[0])*0.5

depomesh = pv.PolyData(surface[:, 3:])
depomesh["radius"] = np.ones(surface[:, 3:].shape[0])*0.5
geom = pv.Box()


# Progress bar is a new feature on master branch
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

l:\program\Lib\site-packages\pyvista\core\filters\data_set.py:2320: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61897/index.html?ui=P_0x20b98d6af90_0&reconnect=auto" class="pyvis…

In [9]:
getdata = testpy.get_pointcloud(etching[:, :, :])

In [6]:
point_cloud = pv.PolyData(getdata[:, 3:])
vectors = getdata[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=1000,
    factor=2,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=5.0, render_points_as_spheres=True)
plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

Widget(value='<iframe src="http://localhost:53729/index.html?ui=P_0x22e255fee10_0&reconnect=auto" class="pyvis…

In [12]:
print(getdata.shape)

print(getdata[10000])

(20530, 6)
[ 2.96875426e-02 -9.05025869e-01  4.24319250e-01  4.90000000e+01
  3.10000000e+01  4.00000000e+01]


In [4]:

# @jit(nopython=True)
def ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, point_cloud):
    # 将光线方向标准化（单位向量）
    ray_direction = ray_direction / np.linalg.norm(ray_direction)

    # 计算从光线起点到每个点的向量
    point_vectors = point_cloud - ray_origin

    # 计算点向量在光线方向上的投影长度
    t = np.dot(point_vectors, ray_direction)

    # 计算光线上投影点的坐标
    projections = ray_origin + np.outer(t, ray_direction)

    # 计算当前点到投影点的距离
    distances = np.linalg.norm(point_cloud - projections, axis=1)

    indice = np.where(distances < 0.5)[0]
    # 如果没有点满足条件，则返回空结果
    if indice.size == 0:
        return None
    near_point = point_cloud[indice]

    distances_near_point = np.linalg.norm(ray_origin - near_point, axis=1)
    # 找到最小距离和最近点
    min_index = np.argmin(distances_near_point)
    closest_point = point_cloud[indice[min_index]]
    # min_distance = distance_in[min_index]
    return closest_point.astype(int)


In [15]:
from joblib import Parallel, delayed

def ray_point_cloud_intersection_parallel(ray_origin, ray_direction, point_cloud, n_jobs=-1, threshold=0.5):
    ray_direction = ray_direction / np.linalg.norm(ray_direction)
    
    def process_batch(batch):
        point_vectors = batch - ray_origin
        t = np.dot(point_vectors, ray_direction)
        projections = ray_origin + t[:, None] * ray_direction
        distances = np.linalg.norm(batch - projections, axis=1)
        valid_indices = np.where(distances < threshold)[0]
        
        if valid_indices.size == 0:
            return None, np.inf
        
        valid_points = batch[valid_indices]
        distances_to_origin = np.linalg.norm(ray_origin - valid_points, axis=1)
        min_index = np.argmin(distances_to_origin)
        
        return valid_points[min_index], distances_to_origin[min_index]
    
    results = Parallel(n_jobs=n_jobs)(delayed(process_batch)(point_cloud[i:i + 1000]) for i in range(0, len(point_cloud), 1000))
    
    closest_point, min_distance = min(results, key=lambda x: x[1])
    return closest_point.astype(int) if min_distance != np.inf else None


In [7]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [18]:
ray_origin = np.array([10.3, 10.1, 160])
ray_direction = np.array([0, 0, -10])

closest_point = ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, getdata[:, 3:])

In [19]:
closest_point

array([ 10,  10, 129])

In [16]:
closest_point = ray_point_cloud_intersection_parallel(ray_origin, ray_direction, getdata[:, 3:])

In [17]:
print(closest_point)

[ 10  10 129]


In [232]:
def find_intersection_face(ray_origin, ray_direction, box_center, box_half_size):
    # 计算立方体的最小角和最大角
    min_corner = box_center - box_half_size
    max_corner = box_center + box_half_size

    # 初始化记录相交时间的数组和相交面的数组
    t_near = -np.inf
    t_far = np.inf
    hit_normal = np.array([0, 0, 0])

    # 遍历三个轴（x, y, z）
    for i in range(3):
        if ray_direction[i] != 0:
            # 计算 t1 和 t2
            t1 = (min_corner[i] - ray_origin[i]) / ray_direction[i]
            t2 = (max_corner[i] - ray_origin[i]) / ray_direction[i]
            
            # 交换 t1 和 t2，如果 t1 > t2
            if t1 > t2:
                t1, t2 = t2, t1
            
            # 更新 t_near 和 t_far
            if t1 > t_near:
                t_near = t1
                hit_normal = np.array([0, 0, 0])
                hit_normal[i] = -1 if ray_direction[i] > 0 else 1
            
            if t2 < t_far:
                t_far = t2

            # 检查是否存在相交
            if t_near > t_far or t_far < 0:
                return None, None  # 不相交

    # 确定哪个面被击中
    if np.array_equal(hit_normal, [-1, 0, 0]):
        return 0  #"Left"
    elif np.array_equal(hit_normal, [1, 0, 0]):
        return 1  #"Right"
    elif np.array_equal(hit_normal, [0, -1, 0]):
        return 2  #"Front"
    elif np.array_equal(hit_normal, [0, 1, 0]):
        return 3  #"Back"
    elif np.array_equal(hit_normal, [0, 0, -1]):
        return 4  #"Bottom"
    elif np.array_equal(hit_normal, [0, 0, 1]):
        return 5  #"Top"
    return None


# 示例使用
ray_origin = np.array([5, 5.1, 5])
ray_direction = np.array([-1, -1, -1])
box_center = np.array([0, 0, 0])
box_half_size = np.array([1, 1, 1])

hit_face = find_intersection_face(ray_origin, ray_direction, box_center, box_half_size)
print(f"Hit face: {hit_face}")

Hit face: 3


In [231]:
# import numpy as np

def find_intersection_face(ray_origin, ray_direction, box_center, box_half_size):
    # 计算立方体的最小角和最大角
    min_corner = box_center - box_half_size
    max_corner = box_center + box_half_size

    # 初始化记录相交时间的数组和相交面的数组
    t_near = -np.inf
    t_far = np.inf
    hit_normal = np.array([0, 0, 0])

    # 遍历三个轴（x, y, z）
    for i in range(3):
        if ray_direction[i] != 0:
            # 计算 t1 和 t2
            t1 = (min_corner[i] - ray_origin[i]) / ray_direction[i]
            t2 = (max_corner[i] - ray_origin[i]) / ray_direction[i]
            
            # 交换 t1 和 t2，如果 t1 > t2
            if t1 > t2:
                t1, t2 = t2, t1
            
            # 更新 t_near 和 t_far
            if t1 > t_near:
                t_near = t1
                hit_normal = np.array([0, 0, 0])
                hit_normal[i] = -1 if ray_direction[i] > 0 else 1
            
            if t2 < t_far:
                t_far = t2

            # 检查是否存在相交
            if t_near > t_far or t_far < 0:
                return None, None  # 不相交

    # 确定哪个面被击中
    hit_face = determine_face(hit_normal)

    return hit_face, t_near

def determine_face(hit_normal):
    # 根据 hit_normal 的方向确定被击中的面
    if np.array_equal(hit_normal, [-1, 0, 0]):
        return "Left"
    elif np.array_equal(hit_normal, [1, 0, 0]):
        return "Right"
    elif np.array_equal(hit_normal, [0, -1, 0]):
        return "Front"
    elif np.array_equal(hit_normal, [0, 1, 0]):
        return "Back"
    elif np.array_equal(hit_normal, [0, 0, -1]):
        return "Bottom"
    elif np.array_equal(hit_normal, [0, 0, 1]):
        return "Top"
    return None

# 示例使用
ray_origin = np.array([5, 5.1, 5])
ray_direction = np.array([-1, -1, -1])
box_center = np.array([0, 0, 0])
box_half_size = np.array([1, 1, 1])

hit_face, t_near = find_intersection_face(ray_origin, ray_direction, box_center, box_half_size)
print(f"Hit face: {hit_face}, t_near: {t_near}")


Hit face: Back, t_near: 4.1


In [220]:
# 示例使用
ray_origin = np.array([0, 10, 0])
ray_direction = np.array([0.0001, -1, 0.001])
box_center = np.array([0, 0, 0])
box_half_size = np.array([1, 1, 1])

hit_face, t_near = find_intersection_face(ray_origin, ray_direction, box_center, box_half_size)
print(f"Hit face: {hit_face}, t_near: {t_near}")

Hit face: Back, t_near: 9.0


In [155]:
print(etching[closest_point[0].astype(int),closest_point[1].astype(int), closest_point[2].astype(int)])

0.0


In [139]:
getdata = testpy.get_pointcloud(etching[:, :, :])

In [ ]:
film_elemnt = np.array([10, 20])

In [156]:
a = np.array((1,2,3,5))
b = np.array((4,5,6))
np.hstack((a,b))

array([1, 2, 3, 5, 4, 5, 6])

In [185]:
react_table = np.array([[[0.700, 0, 1], [0.300, 0, 1]],
                        [[0.200, -1, 0], [0.075, 0, -1]]])

film_elemnt = np.array([10, 20], dtype=float)

def reaction(gas, film):
    # 计算总长度
    total_length = int(film.sum())
    
    # 预分配数组
    reaction_type = np.empty(total_length, dtype=float)
    reaction_rate = np.empty(total_length, dtype=float)
    
    # 填充数组
    current_index = 0
    for i in range(film.shape[0]):
        length = int(film[i])
        reaction_type[current_index:current_index+length] = i
        reaction_rate[current_index:current_index+length] = react_table[gas,i,0]
        current_index += length
    
    reaction_list = np.random.rand(int(film.sum()))

    reaction_selection = reaction_rate > reaction_list
    reaction_choice = np.random.choice(np.where(reaction_selection == True)[0], 1)
    reaction_particle = reaction_type[reaction_choice].astype(int)

    film += react_table[gas, reaction_particle, 1:][0]

    return film

print(reaction(0, film_elemnt))

[10. 21.]


In [641]:
# import numpy as np

# 定义反应表和初始的 film 元素
react_table = np.array([[[0.700, 0, 1], [0.300, 0, 1]],
                        [[0.200, -1, 0], [0.075, 0, -1]]])
film_elemnt = np.array([0.0, 1], dtype=float)

def reaction(gas, film):
    # 计算总长度
    total_length = int(film.sum())

    # 预分配反应类型和反应速率
    reaction_type = np.empty(total_length, dtype=int)
    reaction_rate = np.empty(total_length, dtype=float)

    # 填充反应类型和反应速率
    current_index = 0
    for i in range(film.shape[0]):
        length = int(film[i])
        reaction_type[current_index:current_index + length] = i
        reaction_rate[current_index:current_index + length] = react_table[gas, i, 0]
        current_index += length

    # 生成随机数列以确定反应发生的概率
    reaction_list = np.random.rand(total_length)

    # 筛选出发生反应的粒子
    reaction_indices = np.where(reaction_rate > reaction_list)[0]
    
    # 如果有反应发生，随机选择一个反应
    if reaction_indices.size > 0:
        reaction_choice = np.random.choice(reaction_indices, 1)
        reaction_particle = reaction_type[reaction_choice][0]
        # print(reaction_particle)
        # 更新 film 元素的状态
        film += react_table[gas, reaction_particle, 1:]
        return film, reaction_particle, np.sum(react_table[gas, reaction_particle, 1:]) # film, react_type, etchingDepo 
    else:
        return film, -1, 0


# 调用函数并输出结果
print(reaction(1.0, film_elemnt))


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [638]:
film = np.ones((10, 10, 10, 3))

sumfilm = np.sum(film, axis=-1)
print(sumfilm.shape)

(10, 10, 10)


In [ ]:
    # # 确定哪个面被击中
    # if np.array_equal(hit_normal, [-1, 0, 0]):
    #     return 0  #"Left"
    # elif np.array_equal(hit_normal, [1, 0, 0]):
    #     return 1  #"Right"
    # elif np.array_equal(hit_normal, [0, -1, 0]):
    #     return 2  #"Front"
    # elif np.array_equal(hit_normal, [0, 1, 0]):
    #     return 3  #"Back"
    # elif np.array_equal(hit_normal, [0, 0, -1]):
    #     return 4  #"Bottom"
    # elif np.array_equal(hit_normal, [0, 0, 1]):
    #     return 5  #"Top"
    # return None
ray_origin = np.array([10.3, 10.1, 160])
ray_direction = np.array([0, 0, -10])

closest_point = ray_point_cloud_intersection_vectorized(ray_origin, ray_direction, getdata[:, 3:])


def surface_addition(film, react_point, react, flux):
    film[react_point[0], react_point[1], react_point[2], flux] += react[2]
    film[react_point[0], react_point[1], react_point[2], 6:] += react[0]
    return film

In [246]:
a = np.arange(10)

print(a[:6])
print(a[6:])

[0 1 2 3 4 5]
[6 7 8 9]


In [2]:
a = np.array([0.0, 0.33, 0.5, 0.7, 1.0])
b = np.random.rand()

# 创建布尔掩码，找出满足条件的区间
mask = (b > a[:-1]) & (b <= a[1:])
print(mask)
print(b)
# 找到第一个满足条件的区间索引
indices = np.where(mask)[0]
print(indices[0])
if indices.size > 0:
    index = indices[0]
    print('get')
    print(index)
    print(b)
else:
    print('b is not in any interval')


[False False  True False]
0.5796342470165731
2
get
2
0.5796342470165731


In [278]:
a = np.array([[0.0, 0.33, 0.5, 0.7, 1.0], [0.0, 0.33, 0.5, 0.7, 1.2]])

print(np.where(a > 1.4))

b = np.where(a > 1.4)
print(b[0])
print(b[0].size)

print(a[b])

(array([], dtype=int64), array([], dtype=int64))
[]
0
[]


In [280]:
a = np.array([10, 30, 34, 40, 33])
b = np.array([0, 12, 5, 6, 7])
c = a-b
print(c)

[10 18 29 34 26]


In [286]:
# 原始数组 a
a = np.array([10, 30, 34, 40, 33])

# 总抽取数量 N
N = int(np.sum(a)/2)
print(N)

# 初始化 b 数组
b = np.zeros_like(a)

# 计算总和
total_sum = a.sum()

if total_sum < N:
    raise ValueError("N exceeds the total number of available elements in a.")

# 随机抽取元素
remaining_to_draw = N
while remaining_to_draw > 0:
    # 随机选择一个位置
    idx = np.random.choice(np.where(a > 0)[0])
    # 在选择的位置随机抽取数量，最大为a[idx]和remaining_to_draw之间的最小值
    draw = np.random.randint(0, min(a[idx], remaining_to_draw) + 1)
    # 更新 b 和 a
    b[idx] += draw
    a[idx] -= draw
    remaining_to_draw -= draw

print("随机抽取数量 b:", b)
print("剩余的 a 数组:", a)


73
随机抽取数量 b: [ 0 28 21  4 20]
剩余的 a 数组: [10  2 13 36 13]


In [649]:
a = np.array([10, 30, 34, 40, 33])
# print(a.size)
# 总抽取数量 N
N = 30

# 初始化 b 数组
b = np.zeros_like(a)

# 计算总和
total_sum = a.sum()

if total_sum < N:
    raise ValueError("N exceeds the total number of available elements in a.")

# 随机选择抽取的顺序
indices = np.repeat(np.arange(a.size), a)
# print(indices)
np.random.shuffle(indices)

# 选择前 N 个，统计各类别的抽取数量
unique, counts = np.unique(indices[:N], return_counts=True)
print(unique)
# 更新 b 和 a
b[unique] += counts
a[unique] -= counts

print("随机抽取数量 b:", b)
print("剩余的 a 数组:", a)

[0 1 2 3 4]
随机抽取数量 b: [ 2  5  5 11  7]
剩余的 a 数组: [ 8 25 29 29 26]


In [ ]:
    # # 确定哪个面被击中
    # if np.array_equal(hit_normal, [-1, 0, 0]):
    #     return 0  #"Left"
    # elif np.array_equal(hit_normal, [1, 0, 0]):
    #     return 1  #"Right"
    # elif np.array_equal(hit_normal, [0, -1, 0]):
    #     return 2  #"Front"
    # elif np.array_equal(hit_normal, [0, 1, 0]):
    #     return 3  #"Back"
    # elif np.array_equal(hit_normal, [0, 0, -1]):
    #     return 4  #"Bottom"
    # elif np.array_equal(hit_normal, [0, 0, 1]):
    #     return 5  #"Top"
    # return None

indices = np.repeat(np.arange(a.size), a)
print(indices)
np.random.shuffle(indices)

# 选择前 N 个，统计各类别的抽取数量
unique, counts = np.unique(indices[:N], return_counts=True)

# 更新 b 和 a
b[unique] += counts
a[unique] -= counts

In [578]:
def pickParticles(film, inCell, outFlow):
    # print(film[inCell[0], inCell[1], inCell[2],6:])
    # print(film[inCell[0], inCell[1], inCell[2],6:][0].size)
    indices = np.repeat(np.arange(film[inCell[0], inCell[1], inCell[2],6:][0].size), film[inCell[0], inCell[1], inCell[2],6:][0])
    np.random.shuffle(indices)
    # 选择前 N 个，统计各类别的抽取数量
    print(indices)
    unique, counts = np.unique(indices[:outFlow[0]], return_counts=True) # np.unique(indices[:N+1], return_counts=True)
    print(counts)
    print(unique)
    print('----unique----')
    film[inCell[0], inCell[1], inCell[2], unique+6] -= counts
    return film, unique, counts

In [617]:
def enclose_overflow(film, i, j, k):
    if np.sum(film[i-1, j, k, 6:]) > 30:
        if (np.sum(film[i-1-1, j, k, 6:]) > 0) & (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i-1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i-1, j, k+1, 6:]) > 0):
            inCell = np.array([i-1, j, k])
            overflow = np.sum(film[i-1, j, k, 6:]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i+1, j, k, 6:]) > 30:
        if (np.sum(film[i+1+1, j, k, 6:]) > 0) & (np.sum(film[i+1, j-1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0):
            inCell = np.array([i+1, j, k])
            overflow = np.sum(film[i+1, j, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j-1, k, 6:]) > 30:
        if (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i, j-1-1, k, 6:]) > 0) & (np.sum(film[i+1, j, k, 6:]) > 0) \
            & (np.sum(film[i, j-1, k-1, 6:]) > 0) & (np.sum(film[i, j-1, k+1, 6:]) > 0):
            inCell = np.array([i, j-1, k])
            overflow = np.sum(film[i, j-1, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j+1, k, 6:]) > 30:
        if (np.sum(film[i-1, j+1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) & (np.sum(film[i, j+1+1, k, 6:]) > 0) \
            & (np.sum(film[i, j+1, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0):
            inCell = np.array([i, j+1, k])
            overflow = np.sum(film[i, j+1, k]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j, k-1, 6:]) > 30:
        if (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k-1, 6:]) > 0) \
            & (np.sum(film[i, j, k-1-1, 6:]) > 0) & (np.sum(film[i, j-1, k, 6:]) > 0):
            inCell = np.array([i, j, k-1])
            overflow = np.sum(film[i, j, k-1]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts
    elif np.sum(film[i, j, k+1, 6:]) > 30:
        if (np.sum(film[i-1, j, k+1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0) \
            & (np.sum(film[i, j-1, k, 6:]) > 0) & (np.sum(film[i, j, k+1+1, 6:]) > 0):
            inCell = np.array([i, j, k+1])
            overflow = np.sum(film[i, j, k+1]) - 30
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k][unique+6] += counts

    # less than 30
    elif np.sum(film[i-1, j, k, 6:]) < 30:
        if (np.sum(film[i-1-1, j, k, 6:]) > 0) & (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i-1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i-1, j, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i-1, j, k, 6:])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i-1, j, k][unique+6] += counts
    elif np.sum(film[i+1, j, k, 6:]) < 30:
        if (np.sum(film[i+1+1, j, k, 6:]) > 0) & (np.sum(film[i+1, j-1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) \
            & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i+1, j, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i+1, j, k][unique+6] += counts
    elif np.sum(film[i, j-1, k, 6:]) < 30:
        if (np.sum(film[i-1, j-1, k, 6:]) > 0) & (np.sum(film[i, j-1-1, k, 6:]) > 0) & (np.sum(film[i+1, j, k, 6:]) > 0) \
            & (np.sum(film[i, j-1, k-1, 6:]) > 0) & (np.sum(film[i, j-1, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j-1, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j-1, k][unique+6] += counts
    elif np.sum(film[i, j+1, k, 6:]) < 30:
        if (np.sum(film[i-1, j+1, k, 6:]) > 0) & (np.sum(film[i+1, j+1, k, 6:]) > 0) & (np.sum(film[i, j+1+1, k, 6:]) > 0) \
            & (np.sum(film[i, j+1, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j+1, k])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j+1, k][unique+6] += counts
    elif np.sum(film[i, j, k-1, 6:]) < 30:
        if (np.sum(film[i-1, j, k-1, 6:]) > 0) & (np.sum(film[i+1, j, k-1, 6:]) > 0) & (np.sum(film[i, j+1, k-1, 6:]) > 0) \
            & (np.sum(film[i, j, k-1-1, 6:]) > 0) & (np.sum(film[i, j-1, k, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j, k-1])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k-1][unique+6] += counts
    elif np.sum(film[i, j, k+1, 6:]) < 30:
        if (np.sum(film[i-1, j, k+1, 6:]) > 0) & (np.sum(film[i+1, j, k+1, 6:]) > 0) & (np.sum(film[i, j+1, k+1, 6:]) > 0) \
            & (np.sum(film[i, j-1, k, 6:]) > 0) & (np.sum(film[i, j, k+1+1, 6:]) > 0):
            inCell = np.array([i, j, k])
            overflow = 30 - np.sum(film[i, j, k+1])
            film, unique, counts = pickParticles(film, inCell, overflow)
            film[i, j, k+1][unique+6] += counts
    return film

In [618]:
a = np.array([10, 30, 34, 40, 33])
b = np.zeros(a.shape[0] + 1)
asum = np.sum(a)

print(np.cumsum(a))
# 使用累积和计算 b 的值
b[1:] = np.cumsum(a) / asum

print(b)

[ 10  40  74 114 147]
[0.         0.06802721 0.27210884 0.50340136 0.7755102  1.        ]


In [619]:
# film([x,y,z] Left Right Front Back Bottom Top, type1, type2, ...)

def surface_advancement(film, type):
    N = int(30)
    sumfilm = np.sum(film[:, :, :, 6:], axis=-1, dtype=int)
    print(sumfilm[:, :, 2])
    if type == 1:
        upperCell = np.where(sumfilm > 2*N)
        # lowerCell = np.where(sumfilm < N/5)
        # upperCell = sumfilm > 2*N
        # lowerCell = sumfilm < N/5
        print(upperCell)
        # print(lowerCell)
        if upperCell[0].size == 0:
            return film
        else:
            outFlowParticles = sumfilm[upperCell] - N
            flux = film[upperCell[0], upperCell[1], upperCell[2], :6][0]
            print(flux)
            print('----flux----')
            flux_rate = np.zeros(flux.shape[0] + 1)
            flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
            flow_random = np.random.rand()
            mask = (flow_random > flux_rate[:-1]) & (flow_random <= flux_rate[1:])
            print(mask)
            flow_choice = np.where(mask)[0]
            print(flow_choice)
            # indices = np.repeat(np.arange(film[upperCell][6:].size), film[upperCell][6:])
            # np.random.shuffle(indices)
            # # 选择前 N 个，统计各类别的抽取数量
            # unique, counts = np.unique(indices[:N+1], return_counts=True)
            # film[upperCell[0], upperCell[1], upperCell[2]][unique+6]     -= counts
            film, unique, counts = pickParticles(film, upperCell, outFlowParticles)
            if flow_choice == 0:
                film[upperCell[0] - 1, upperCell[1], upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0] - 1, upperCell[1], upperCell[2])
            elif flow_choice == 1:
                film[upperCell[0] + 1, upperCell[1], upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0] + 1, upperCell[1], upperCell[2])
            elif flow_choice == 2:
                film[upperCell[0], upperCell[1] - 1, upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1] - 1, upperCell[2])
            elif flow_choice == 3:
                film[upperCell[0], upperCell[1] + 1, upperCell[2],unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1] + 1, upperCell[2])
            elif flow_choice == 4:
                film[upperCell[0], upperCell[1], upperCell[2] - 1,unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1], upperCell[2] - 1)
            elif flow_choice == 5:
                film[upperCell[0], upperCell[1], upperCell[2] + 1,unique+6] += counts
                film = enclose_overflow(film, upperCell[0], upperCell[1], upperCell[2] + 1)
            else:
                print('error flow')
        return film
    elif type == -1:
        # lowerCell = np.where(sumfilm > 2*N)
        lowerCell = np.where(sumfilm < N/5)
        # lowerCell = sumfilm > 2*N
        # lowerCell = sumfilm < N/5
        print(lowerCell)
        # print(lowerCell)
        if lowerCell.size == 0:
            return film
        else:
            outFlowParticles = sumfilm[lowerCell]
            flux = film[lowerCell[0], lowerCell[1], lowerCell[2], :6][0]
            flux_min = np.min(flux)
            flux -= flux_min
            flux_rate = np.zeros(flux.shape[0] + 1)
            flux_rate[1:] = np.cumsum(flux)/np.sum(flux)
            flow_random = np.random.rand()
            mask = (flow_random > flux_rate[:-1]) & (flow_random <= flux_rate[1:])
            flow_choice = np.where(mask)[0]
            counts = film[lowerCell[0], lowerCell[1], lowerCell[2], 6:]
            if flow_choice == 0:
                film[lowerCell[0] - 1, lowerCell[1], lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0] - 1, lowerCell[1], lowerCell[2])
            elif flow_choice == 1:
                film[lowerCell[0] + 1, lowerCell[1], lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0] + 1, lowerCell[1], lowerCell[2])
            elif flow_choice == 2:
                film[lowerCell[0], lowerCell[1] - 1, lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1] - 1, lowerCell[2])
            elif flow_choice == 3:
                film[lowerCell[0], lowerCell[1] + 1, lowerCell[2],unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1] + 1, lowerCell[2])
            elif flow_choice == 4:
                film[lowerCell[0], lowerCell[1], lowerCell[2] - 1,unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1], lowerCell[2] - 1)
            elif flow_choice == 5:
                film[lowerCell[0], lowerCell[1], lowerCell[2] + 1,unique+6] += counts
                film = enclose_overflow(film, lowerCell[0], lowerCell[1], lowerCell[2] + 1)
            else:
                print('error flow')
        return film   

            

In [620]:
testfilm = np.zeros((6, 6, 4, 10), dtype=int)

a = np.random.randint(1, 16, size=(6, 6, 4, 4))

testfilm[:, :, :, 6:] = a
print(testfilm)

[[[[ 0  0  0 ... 12  2 10]
   [ 0  0  0 ... 12  5 12]
   [ 0  0  0 ...  8 10 12]
   [ 0  0  0 ...  1  6 11]]

  [[ 0  0  0 ...  5 11 11]
   [ 0  0  0 ... 10  2  7]
   [ 0  0  0 ...  6 15 12]
   [ 0  0  0 ...  4  2  8]]

  [[ 0  0  0 ... 14 12  2]
   [ 0  0  0 ...  5 14  9]
   [ 0  0  0 ...  6  4  4]
   [ 0  0  0 ... 14  1  8]]

  [[ 0  0  0 ...  4  3  7]
   [ 0  0  0 ...  5  4  1]
   [ 0  0  0 ...  9  1  3]
   [ 0  0  0 ... 11 12  1]]

  [[ 0  0  0 ... 10 13  2]
   [ 0  0  0 ...  7 11  7]
   [ 0  0  0 ...  8  2  3]
   [ 0  0  0 ...  5 13  7]]

  [[ 0  0  0 ...  6  6  6]
   [ 0  0  0 ...  4 14  4]
   [ 0  0  0 ...  3 12  4]
   [ 0  0  0 ...  1 15  5]]]


 [[[ 0  0  0 ... 14  6  4]
   [ 0  0  0 ...  1 15  3]
   [ 0  0  0 ...  4 13  1]
   [ 0  0  0 ...  3 14  8]]

  [[ 0  0  0 ...  1 15 12]
   [ 0  0  0 ...  9  3  6]
   [ 0  0  0 ...  3  9  7]
   [ 0  0  0 ...  4  6  9]]

  [[ 0  0  0 ...  2  2  9]
   [ 0  0  0 ...  3  7 14]
   [ 0  0  0 ... 15  7  5]
   [ 0  0  0 ...  6  3 11]]

  [[ 0  

In [621]:
print(np.sum(testfilm, axis=-1))

[[[37 38 39 27]
  [35 24 41 20]
  [36 40 16 26]
  [18 11 16 26]
  [31 37 24 34]
  [21 32 20 27]]

 [[31 33 23 38]
  [40 31 32 29]
  [27 31 32 30]
  [27 44 41 37]
  [33  9 36 22]
  [20 19 26 31]]

 [[32 32 29 24]
  [ 7 26 45 41]
  [36 29 48 15]
  [17 20 28 25]
  [37 39 37 45]
  [36 31 19 41]]

 [[32 24 30 50]
  [32 17 36 40]
  [47 30 28 44]
  [41 40 26 30]
  [33 40 40 42]
  [42 20 26 45]]

 [[33 39 47 22]
  [47 41 38 29]
  [38 22 33 36]
  [37 32 23 17]
  [39 34 17 33]
  [26 38 37 35]]

 [[31 21 28 40]
  [24 26 51 46]
  [23 40 26 24]
  [29 28 27 25]
  [24 28 16 38]
  [26 41 40 35]]]


In [622]:
testfilm[:, :, 3, :] = 0

In [623]:
testfilm[1, 1, 2, -1] = 60
testfilm[1, 1, 2, 1] = 40
print(np.sum(testfilm[1, 1, 2], axis=-1))
print(testfilm[1, 1, 2])

125
[ 0 40  0  0  0  0 13  3  9 60]


In [624]:
print(np.sum(testfilm[:, :, 3, 6:], axis=-1))
print(np.sum(testfilm[:, :, 2, 6:], axis=-1))
print(np.sum(testfilm[:, :, 1, 6:], axis=-1))
print(np.sum(testfilm[:, :, 0, 6:], axis=-1))

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[39 41 16 16 24 20]
 [23 85 32 41 36 26]
 [29 45 48 28 37 19]
 [30 36 28 26 40 26]
 [47 38 33 23 17 37]
 [28 51 26 27 16 40]]
[[38 24 40 11 37 32]
 [33 31 31 44  9 19]
 [32 26 29 20 39 31]
 [24 17 30 40 40 20]
 [39 41 22 32 34 38]
 [21 26 40 28 28 41]]
[[37 35 36 18 31 21]
 [31 40 27 27 33 20]
 [32  7 36 17 37 36]
 [32 32 47 41 33 42]
 [33 47 38 37 39 26]
 [31 24 23 29 24 26]]


In [625]:

filmin = surface_advancement(testfilm, 1)

[[39 41 16 16 24 20]
 [23 85 32 41 36 26]
 [29 45 48 28 37 19]
 [30 36 28 26 40 26]
 [47 38 33 23 17 37]
 [28 51 26 27 16 40]]
(array([1], dtype=int64), array([1], dtype=int64), array([2], dtype=int64))
[ 0 40  0  0  0  0]
----flux----
[False  True False False False False]
[1]
[3 3 3 3 3 0 3 3 3 2 0 3 0 3 0 0 3 0 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3
 3 3 3 2 3 1 3 3 3 0 3 3 3 3 3 0 3 2 3 2 3 3 1 0 1 2 3 2 3 3 3 3 0 3 3 3 3
 0 3 3 2 2 3 0 3 3 0 3]
[ 8  1  4 42]
[0 1 2 3]
----unique----


In [626]:
print(np.sum(filmin[:, :, 3, 6:], axis=-1))
print(np.sum(filmin[:, :, 2, 6:], axis=-1))
print(np.sum(filmin[:, :, 1, 6:], axis=-1))
print(np.sum(filmin[:, :, 0, 6:], axis=-1))

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[ 39  41  16  16  24  20]
 [ 23  30  32  41  36  26]
 [ 29 100  48  28  37  19]
 [ 30  36  28  26  40  26]
 [ 47  38  33  23  17  37]
 [ 28  51  26  27  16  40]]
[[38 24 40 11 37 32]
 [33 31 31 44  9 19]
 [32 26 29 20 39 31]
 [24 17 30 40 40 20]
 [39 41 22 32 34 38]
 [21 26 40 28 28 41]]
[[37 35 36 18 31 21]
 [31 40 27 27 33 20]
 [32  7 36 17 37 36]
 [32 32 47 41 33 42]
 [33 47 38 37 39 26]
 [31 24 23 29 24 26]]


In [614]:
filmin[1, 1, 2]

array([ 0, 40,  0,  0,  0,  0,  3,  1,  4, 22])

In [615]:
filmin[2, 1, 2]

array([ 0,  0,  0,  0,  0,  0, 12,  7, 14, 48])